In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

from pyomo.network import Arc, SequentialDecomposition

from idaes.core.util.initialization import propagate_state

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

import pandas as pd

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog
from pyomo.opt import TerminationCondition, SolverStatus

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.generic_models.unit_models import Flash

In [2]:
from R32_bmimPF6_PR import configuration
data = pd.read_csv('R32IL.csv')

solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [3]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(323.15)
m.fs.F101.inlet.pressure.fix(399800)
m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(0.5)


# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'bmimPF6'].unfix()

R32_x = np.zeros((len(data)))
P = np.zeros((len(data)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "bmimPF6"] == m.fs.liq)

bin1 = np.linspace(-0.5, 0.5, 10)
bin2 = np.linspace(-0.5, 0.5, 10)

b1 = []
b2 = []
Pcalc= []
Temp = []
R32x = []
P = np.zeros((len(data), len(bin1), len(bin2)))

for j in range(len(bin1)):
    m.fs.properties.PR_kappa['bmimPF6', 'R32'].fix(bin1[j])
    print(bin1[j])
        
    for k in range(len(bin2)):
        m.fs.properties.PR_kappa['R32', 'bmimPF6'].fix(bin2[k])
        print(bin2[k])
        
        for i in range(len(data)):
            m.fs.liq = data["x_bmimPF6"].iloc[i]
            m.fs.F101.inlet.temperature.fix(float(data["temperature"].iloc[i]))
            m.fs.F101.inlet.pressure.fix(float(data["pressure"].iloc[i]))
            m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(float(data["x_R32"].iloc[i])+0.1)
            m.fs.F101.inlet.mole_frac_comp[0,'bmimPF6'].fix(float(1-(data["x_R32"].iloc[i]+0.1)))
            m.fs.F101.vap_outlet.temperature.fix(float(data["temperature"].iloc[i]))

            try:
                m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
                
                status = solver.solve(m, tee = False)
                
                if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):

                    b1.append(bin1[j])
                    b2.append(bin2[k])
                    Pcalc.append(value(m.fs.F101.vap_outlet.pressure[0]))
                    Temp.append(value(m.fs.F101.vap_outlet.temperature[0]))
                    P[i,j,k] = value(m.fs.F101.vap_outlet.pressure[0])
                    R32x.append(value(m.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]))
                else:
                    P[i,j,k] = "NaN"

            except ValueError:
                P[i,j,k] = "NaN"


-0.5
-0.5
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
    

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termin

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termin

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
-0.05555555555555558
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.n

    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="fs.F101"

...

    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="fs.F101";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBl

    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="fs.F101";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock"

    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="fs.F101";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
          Exceeded.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination 

    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="fs.F101";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
          Exceeded.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination cond

    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="SimpleBlock";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="fs.F101";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
          infeasible point. Problem may be infeasible.


In [4]:
# np.set_printoptions(threshold=np.inf)
# print(P)


In [5]:
# plt.plot(R32_x,P,"*",label="IDAES")  
# plt.plot(data["x_R32"],data["pressure"],"*",label="data")  

# plt.ylabel('Pressure (Pa)')
# plt.xlabel('x R32')
# plt.grid(True)
# plt.legend()
# plt.show()

In [6]:
# m.fs.F101.report()

In [7]:
b1d = pd.DataFrame(b1)
b2d = pd.DataFrame(b2)
p1d = pd.DataFrame(Pcalc)
Td = pd.DataFrame(Temp)
Rd = pd.DataFrame(R32x)

In [8]:
result1 = pd.concat([b1d, b2d], axis=1)
result1 = pd.concat([result1, p1d], axis=1)
result1 = pd.concat([result1, Td], axis=1)
result1 = pd.concat([result1, Rd], axis=1)
print(result1)

       0         0              0      0       0
0   -0.5 -0.500000     405.536774  348.2  0.0030
1   -0.5 -0.388889  344538.703686  283.2  0.8150
2   -0.5 -0.388889     797.920422  322.8  0.0060
3   -0.5 -0.388889     930.820015  348.2  0.0030
4   -0.5 -0.277778    1530.566939  283.2  0.0206
..   ...       ...            ...    ...     ...
959  0.5  0.277778  353816.424958  283.2  0.0206
960  0.5  0.277778  368373.441632  298.1  0.0180
961  0.5  0.277778  153639.505245  322.8  0.0060
962  0.5  0.277778   92857.160811  348.2  0.0030
963  0.5  0.277778  940123.561120  348.2  0.0290

[964 rows x 5 columns]


In [9]:
result1.to_csv('sens2.csv')

In [10]:
print(result1)

       0         0              0      0       0
0   -0.5 -0.500000     405.536774  348.2  0.0030
1   -0.5 -0.388889  344538.703686  283.2  0.8150
2   -0.5 -0.388889     797.920422  322.8  0.0060
3   -0.5 -0.388889     930.820015  348.2  0.0030
4   -0.5 -0.277778    1530.566939  283.2  0.0206
..   ...       ...            ...    ...     ...
959  0.5  0.277778  353816.424958  283.2  0.0206
960  0.5  0.277778  368373.441632  298.1  0.0180
961  0.5  0.277778  153639.505245  322.8  0.0060
962  0.5  0.277778   92857.160811  348.2  0.0030
963  0.5  0.277778  940123.561120  348.2  0.0290

[964 rows x 5 columns]


In [11]:
# result1 = pd.concat([result1, Rd], axis=1)
# print(result1)

In [12]:
# print(Rd)

In [13]:
# print(R32x)